In [1]:
!pip install transformers
!pip install libwapiti parsivar
!pip show libwapiti
!pip install tensorflow
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 4.2 MB 5.1 MB/s 
     |████████████████████████████████| 6.6 MB 39.4 MB/s 
     |████████████████████████████████| 86 kB 5.0 MB/s 
     |████████████████████████████████| 596 kB 47.7 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 233 kB 5.1 MB/s 
     |████████████████████████████████| 36.2 MB 1.4 MB/s 
     |████████████████████████████████| 1.5 MB 43.9 MB/s 
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154833 sha256=6a92c43847afe0a046262b8044b97e2b9901a6b28825f93af2a326f333cfe09b
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a

In [5]:
import pandas as pd
from transformers import MarianMTModel, MarianConfig
from transformers import MarianTokenizer
from parsivar import *
import pprint
from tokenizers import Tokenizer
from tokenizers.models import BPE

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
tokenizer = Tokenizer(BPE())

In [ ]:
!cp /content/drive/MyDrive/NLP/CA05/data.zip /content/data.zip
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


In [ ]:
from tokenizers.trainers import BpeTrainer

trainer = BpeTrainer(special_tokens=["[UNK]", "[CLS]", "[SEP]", "[PAD]", "[MASK]"])
tokenizer.train(files=["data/train.en", "data/train.fa"], trainer=trainer)

In [ ]:
output = tokenizer.encode("Hello, y'all! How are you 😁 ?")
print(output.tokens)

['ell', 'o', ', ', 'y', "'", 'all', '! ', 'ow ', 'are you ', ' ', '?']


In [ ]:
text_files = ['test.en', 'test.fa', 'train.en', 'train.fa', 'valid.en', 'valid.fa']
for fname in text_files:
  f = open('data/'+fname)
  tlist = f.readlines()
  tlist = [tokenizer.encode(x) for x in tlist]

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i data/train.en -o data/en.code -s 10000

In [ ]:
!python OpenNMT-py/tools/learn_bpe.py -i data/train.fa -o data/fa.code -s 10000

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c data/en.code -i data/train.en -o data/train.en.bpe
!python OpenNMT-py/tools/apply_bpe.py -c data/en.code -i data/test.en -o data/test.en.bpe
!python OpenNMT-py/tools/apply_bpe.py -c data/en.code -i data/valid.en -o data/valid.en.bpe

In [ ]:
!python OpenNMT-py/tools/apply_bpe.py -c data/fa.code -i data/train.fa -o data/train.fa.bpe
!python OpenNMT-py/tools/apply_bpe.py -c data/fa.code -i data/test.fa -o data/test.fa.bpe
!python OpenNMT-py/tools/apply_bpe.py -c data/fa.code -i data/valid.fa -o data/valid.fa.bpe

In [ ]:
def load_and_pre_processing(englishPath,persianPath):
  f = open(englishPath,'r')
  eng = f.readlines()
  f.close()
  f = open(persianPath,'r')
  per = f.readlines()
  # per = [normalizer.normalize(x) for x in per]
  f.close()

  return pd.DataFrame.from_dict({'eng':eng, 'per':per})

In [ ]:
trainDf2 = load_and_pre_processing('/content/data/train.en', '/content/data/train.fa')
trainDf = load_and_pre_processing('/content/data/train.en.bpe', '/content/data/train.fa.bpe')
validDf = load_and_pre_processing('/content/data/valid.en.bpe', '/content/data/valid.fa.bpe')
testDf = load_and_pre_processing('/content/data/test.en.bpe', '/content/data/test.fa.bpe')

# openNMT

In [16]:
!cp /content/drive/MyDrive/data.zip /content/data.zip
!unzip /content/data.zip

Archive:  /content/data.zip
   creating: data/
  inflating: data/test.en            
  inflating: data/test.fa            
  inflating: data/train.en           
  inflating: data/train.fa           
  inflating: data/valid.en           
  inflating: data/valid.fa           


In [3]:
!git clone -b r1 https://github.com/OpenNMT/OpenNMT-tf.git

Cloning into 'OpenNMT-tf'...
remote: Enumerating objects: 34424, done.
remote: Counting objects: 100% (6084/6084), done.
remote: Compressing objects: 100% (797/797), done.
remote: Total 34424 (delta 5299), reused 5915 (delta 5225), pack-reused 28340
Receiving objects: 100% (34424/34424), 23.46 MiB | 22.16 MiB/s, done.
Resolving deltas: 100% (28961/28961), done.


In [1]:
!pip install OpenNMT-tf[tensorflow_gpu]==1.*

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!python OpenNMT-tf/third_party/learn_bpe.py -i data/train.en -o data/en.code -s 10000

In [3]:
!python OpenNMT-tf/third_party/learn_bpe.py -i data/train.fa -o data/fa.code -s 10000

In [4]:
!python OpenNMT-tf/third_party/apply_bpe.py -c  data/en.code -i  /content/data/train.en -o /content/data/train.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c  data/en.code -i  /content/data/valid.en -o /content/data/valid.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/en.code -i /content/data/test.en -o /content/data/test.en.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i /content/data/train.fa -o /content/data/train.fa.bpe
!python OpenNMT-tf/third_party/apply_bpe.py -c data/fa.code -i /content/data/valid.fa -o /content/data/valid.fa.bpe

In [5]:
!onmt-build-vocab --size 50000 --save_vocab /content/data/en-vocab.txt /content/data/train.en.bpe

In [6]:
!onmt-build-vocab --size 50000 --save_vocab /content/data/fa-vocab.txt /content/data/train.fa.bpe

In [22]:
!cp /content/drive/MyDrive/data.yaml /content/data.yml

In [ ]:
!nvidia-smi

Sun Jun  5 19:37:03 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   39C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!onmt-main train_and_eval --model_type Transformer --config data.yml --num_gpus 1








2022-06-05 19:37:05.890973: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.



INFO:tensorflow:Using parameters:
data:
  eval_features_file: /content/data/valid.en.bpe
  eval_labels_file: /content/data/valid.fa.bpe
  source_words_vocabulary: /content/data/en-vocab.txt
  target_words_vocabulary: /content/data/fa-vocab.txt
  train_features_file: /content/data/train.en.bpe
  train_labels_file: /content/data/train.fa.bpe
eval:
  batch_size: 30
  eval_delay: 1800
  export_format: saved_model
  export_on_best: bleu
  exporters: last
  length_bucket_width: 5
  max_exports_to_keep: 15
  save_eval_predictions: true
  scorers: bleu, rouge, wer, ter, prf
  steps: 1000
infer:
  batch_size: 128
  bucket_width: null
model_dir: /content/drive/MyDrive/run/
params:
  beam_width: 5
  clip_gradients: 5.0
  decay_params:
    decay_rate: 0.7
    decay_steps: 100000
  decay_type: exponential_decay
 

In [ ]:
# ادامه پردازش تا مرحله آخر یعنی گام 50،000 در این فایل موجود است و در ادامه شاهد ارزیابی سیستم هستیم

In [2]:
!onmt-main infer --auto_config    --config data.yml  --features_file data/test.en --predictions_file data/out.fa








INFO:tensorflow:Loading model description from /content/drive/MyDrive/run/model_description.py
2022-06-10 18:06:04.794983: W tensorflow/python/util/util.cc:299] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


Traceback (most recent call last):
  File "/usr/local/bin/onmt-main", line 8, in <module>
    sys.exit(main())
  File "/usr/local/lib/python3.7/dist-packages/opennmt/bin/main.py", line 169, in main
    hvd=hvd)
  File "/usr/local/lib/python3.7/dist-packages/opennmt/runner.py", line 96, in __init__
    self._model.initialize(self._config["data"], params=self._config["params"])
  File "/usr/local/lib/python3.7/dist-packages/opennmt/models/sequence_to_sequence.py", line 162, in initialize
    super(SequenceToSequence, self).initialize(metadata, params=params)
  File "/usr/local/lib/python3.7/dist-packages/opennmt/models/model.py", line 72, in initialize
    self.examples_inputter.initialize(metadata)
  File "/u

In [ ]:
!sed -i "s/@@ //g" data/out.fa

In [ ]:
#این امتیازی  است که مدل برای
# bleu
# بدست آورده است
!perl  OpenNMT-tf/third_party/multi-bleu-detok.perl data/test.fa < data/out.fa

BLEU = 0.83, 28.0/2.8/0.5/0.1 (BP=0.611, ratio=0.670, hyp_len=7629, ref_len=11385)


In [8]:
#ولی در ادامه برای ارزیابی از کتابخانه
# nltk
# استفاده کردیم
from nltk.translate.bleu_score import corpus_bleu
from nltk.translate.chrf_score import corpus_chrf
from nltk.translate.gleu_score import corpus_gleu
from nltk.translate.nist_score import corpus_nist
from nltk.translate.ribes_score import corpus_ribes
import numpy as np

In [ ]:
f = open('data/test.fa','r')
testList = f.readlines()
f = open('data/out.fa','r')
outList = f.readlines()
testList2 = [[x] for x in testList]
# testList = [x.split() for x in testList]
# outList = [x.split() for x in outList]

In [ ]:
corpus_bleu(testList2, outList, weights=(0.1, 0.2, 0.3, 0.4),
                smoothing_function=None, auto_reweigh=False,
                emulate_multibleu=False)

0.159860174400443

In [ ]:
corpus_bleu(testList2, outList, weights=(0.25, 0.25, 0.25, 0.25),
                smoothing_function=None, auto_reweigh=False,
                emulate_multibleu=False)

0.2189175273050558

In [ ]:
corpus_chrf(testList, outList, min_len=1, max_len=6, beta=3.0)

0.19898926089703095

In [ ]:
corpus_gleu(testList2, outList, min_len=1, max_len=4)

0.29907769335500023

In [ ]:
corpus_nist(testList2, outList, n=4)

3.2679634120617625e-05

In [5]:
!cp -r /content/drive/MyDrive/run/eval eval

In [6]:
!sed -i "s/@@ //g" /content/eval/predictions.txt.10000
!sed -i "s/@@ //g" /content/eval/predictions.txt.2500
!sed -i "s/@@ //g" /content/eval/predictions.txt.17500
!sed -i "s/@@ //g" /content/eval/predictions.txt.20000
!sed -i "s/@@ //g" /content/eval/predictions.txt.27500
!sed -i "s/@@ //g" /content/eval/predictions.txt.35000
!sed -i "s/@@ //g" /content/eval/predictions.txt.37500
!sed -i "s/@@ //g" /content/eval/predictions.txt.45000
!sed -i "s/@@ //g" /content/eval/predictions.txt.50000

In [11]:
outsteps = [2500, 10000, 17500, 20000, 27500, 35000, 37500, 45000, 50000]
for step in outsteps:
  f = open('data/valid.fa','r')
  testList = f.readlines()
  f = open('eval/predictions.txt.' + str(step),'r')
  outList = f.readlines()
  testList = testList[:len(outList)]
  testList2 = [[x] for x in testList]
  print(step,': ')
  print('bleu:\t', corpus_bleu(testList2, outList, weights=(0.1, 0.2, 0.3, 0.4),smoothing_function=None, auto_reweigh=False))
  print('bleu*:\t', corpus_bleu(testList2, outList, weights=(0.25, 0.25, 0.25, 0.25),smoothing_function=None, auto_reweigh=False))
  print('chrf:\t', corpus_chrf(testList, outList, min_len=1, max_len=6, beta=3.0))
  print('gleu:\t', corpus_gleu(testList2, outList, min_len=1, max_len=4))
  print('nist:\t', corpus_nist(testList2, outList, n=4))


2500 : 
bleu:	 0.0960501930236401
bleu*:	 0.1452839345249823
chrf:	 0.1088837782806987
gleu:	 0.17626440677966101
nist:	 2.855122949612588
10000 : 
bleu:	 0.06400196151180329
bleu*:	 0.08817451800604495
chrf:	 0.11000981946688534
gleu:	 0.17472549612168833
nist:	 0.31509807509277804
17500 : 
bleu:	 0.1686483146595182
bleu*:	 0.22796411064704797
chrf:	 0.16390148814663225
gleu:	 0.28932289961364677
nist:	 3.676347574667495
20000 : 
bleu:	 0.19172154236215697
bleu*:	 0.2595637979247066
chrf:	 0.1779611546538572
gleu:	 0.3112521528353639
nist:	 4.562611430311891
27500 : 
bleu:	 0.11882793890754093
bleu*:	 0.16192225864228132
chrf:	 0.1353158925217216
gleu:	 0.23237528906508093
nist:	 1.7618423140511477
35000 : 
bleu:	 0.174980578067585
bleu*:	 0.23603653618534004
chrf:	 0.1731455646778231
gleu:	 0.2976651779850256
nist:	 3.8806335770813885
37500 : 
bleu:	 0.1808759911011292
bleu*:	 0.24373516656743688
chrf:	 0.1716808924232156
gleu:	 0.30334563988630053
nist:	 4.078190570111025
45000 : 
b